In [1]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
import numpy as np

In [2]:
# nltk.download('punkt')
# nltk.download('stopwords')

Read the Data

In [3]:
twitter_2013dev_a = pd.read_csv('dataset/twitter-2013dev-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2013test_a = pd.read_csv('dataset/twitter-2013test-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2013train_a = pd.read_csv('dataset/twitter-2013train-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2014sarcasm_a = pd.read_csv('dataset/twitter-2014sarcasm-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2014test_a = pd.read_csv('dataset/twitter-2014test-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2015test_a = pd.read_csv('dataset/twitter-2015test-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2015train_a = pd.read_csv('dataset/twitter-2015train-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2016dev_a = pd.read_csv('dataset/twitter-2016dev-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2016devtest_a = pd.read_csv('dataset/twitter-2016devtest-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])
twitter_2016test_a = pd.read_csv('dataset/twitter-2016test-A.txt', delimiter="\t", names=["id", "sentiment", "sentence", "dates"])
twitter_2016train_a = pd.read_csv('dataset/twitter-2016train-A.txt', delimiter="\t", names=["id", "sentiment", "sentence"])

data_df = pd.DataFrame()
# data_df = pd.concat([twitter_2013dev_a, twitter_2013test_a, twitter_2013train_a, twitter_2014sarcasm_a, twitter_2014test_a,
#                      twitter_2015test_a, twitter_2015train_a, twitter_2016dev_a, twitter_2016devtest_a,
#                      twitter_2016test_a[["id", "sentiment", "sentence"]], twitter_2016train_a])

data_df = pd.concat([twitter_2013dev_a, twitter_2013test_a, twitter_2013train_a])
data_df.head()

id sentiment  \
0  260097528899452929   neutral   
1  263791921753882624   neutral   
2  264194578381410304  negative   
3  264041328420204544   neutral   
4  263816256640126976   neutral   

                                            sentence  
0  Won the match #getin . Plus\u002c tomorrow is ...  
1  Some areas of New England could see the first ...  
2  @francesco_con40 2nd worst QB. DEFINITELY Tony...  
3  #Thailand Washington - US President Barack Oba...  
4  Did y\u2019all hear what Tony Romo dressed up ...

Check for null values on the data

In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14885 entries, 0 to 9683
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         14885 non-null  int64 
 1   sentiment  14885 non-null  object
 2   sentence   14885 non-null  object
dtypes: int64(1), object(2)
memory usage: 465.2+ KB


In [5]:
data_df.isnull().values.any()

False

Drop the ID column and reset the indexes

In [6]:
data_df.drop("id", axis=1, inplace=True)
data_df.reset_index(inplace=True, drop=True)
data_df.head()

sentiment                                           sentence
0   neutral  Won the match #getin . Plus\u002c tomorrow is ...
1   neutral  Some areas of New England could see the first ...
2  negative  @francesco_con40 2nd worst QB. DEFINITELY Tony...
3   neutral  #Thailand Washington - US President Barack Oba...
4   neutral  Did y\u2019all hear what Tony Romo dressed up ...

In [7]:
def remove_irr_char(sentence):
    """
        This function removes irrelevant characters such as punctuations and etc.
        Accepts a string sentence
        Returns a string cleaned sentence
    """
    word_list = []
    tokenized_sent = nltk.word_tokenize(sentence)
    for i in tokenized_sent:
        word_list.append(''.join(j for j in i if i.isalnum()))

    word_list = list(filter(None,  word_list))

    return ' '.join(word_list).lower()

In [8]:
data_df["cleaned_sentence"] = data_df.apply(lambda x: remove_irr_char(x[1]), axis=1)
data_df.head()

sentiment                                           sentence  \
0   neutral  Won the match #getin . Plus\u002c tomorrow is ...   
1   neutral  Some areas of New England could see the first ...   
2  negative  @francesco_con40 2nd worst QB. DEFINITELY Tony...   
3   neutral  #Thailand Washington - US President Barack Oba...   
4   neutral  Did y\u2019all hear what Tony Romo dressed up ...   

                                    cleaned_sentence  
0  won the match getin tomorrow is a very busy wi...  
1  some areas of new england could see the first ...  
2  2nd worst qb definitely tony romo the man who ...  
3  thailand washington us president barack obama ...  
4  did hear what tony romo dressed up as for hall...

In [9]:
def remove_stop_words(sentence):
    """
        This function removes stop words from the cleaned sentence
        Accepts a string
        Returns a string without the stop words
    """
    tokenized = nltk.word_tokenize(sentence)
    stop_words = set({"a", "about", "an", "are", "as", "at", "be", "by", "com", "de", "en", "for", "from", "how", "i", "in",
                      "is", "it", "la", "of", "on", "or", "that", "this", "to", "was", "what", "when", "where", "who", "will",
                      "with", "und", "the", "www"})

    filtered_list = [
        word for word in tokenized if word.casefold() not in stop_words
    ]

    return ' '.join(filtered_list)

In [10]:
data_df["stopwords_remove_sentence"] = data_df.apply(lambda x: remove_stop_words(x[2]), axis=1)
data_df.head()

sentiment                                           sentence  \
0   neutral  Won the match #getin . Plus\u002c tomorrow is ...   
1   neutral  Some areas of New England could see the first ...   
2  negative  @francesco_con40 2nd worst QB. DEFINITELY Tony...   
3   neutral  #Thailand Washington - US President Barack Oba...   
4   neutral  Did y\u2019all hear what Tony Romo dressed up ...   

                                    cleaned_sentence  \
0  won the match getin tomorrow is a very busy wi...   
1  some areas of new england could see the first ...   
2  2nd worst qb definitely tony romo the man who ...   
3  thailand washington us president barack obama ...   
4  did hear what tony romo dressed up as for hall...   

                           stopwords_remove_sentence  
0  won match getin tomorrow very busy awareness a...  
1  some areas new england could see first flakes ...  
2  2nd worst qb definitely tony romo man likes sh...  
3  thailand washington us president barack obama ...  
4  did hear tony romo dressed up halloween giants...

In [11]:
stemmer = PorterStemmer()

def stem_tokens(sentence):
    """
        This function stems the sentence to convert each tokens to their root base
        Accepts a string
        Returns a string sentence in their root base
    """
    word_list = []
    tokenized_sent = nltk.word_tokenize(sentence)
    for i in tokenized_sent:
        stem_word = stemmer.stem(i)
        word_list.append(stem_word)

    return ' '.join(word_list)

In [12]:
data_df["stemmed_sentence"] = data_df.apply(lambda x: stem_tokens(x[3]), axis=1)
data_df.head()

sentiment                                           sentence  \
0   neutral  Won the match #getin . Plus\u002c tomorrow is ...   
1   neutral  Some areas of New England could see the first ...   
2  negative  @francesco_con40 2nd worst QB. DEFINITELY Tony...   
3   neutral  #Thailand Washington - US President Barack Oba...   
4   neutral  Did y\u2019all hear what Tony Romo dressed up ...   

                                    cleaned_sentence  \
0  won the match getin tomorrow is a very busy wi...   
1  some areas of new england could see the first ...   
2  2nd worst qb definitely tony romo the man who ...   
3  thailand washington us president barack obama ...   
4  did hear what tony romo dressed up as for hall...   

                           stopwords_remove_sentence  \
0  won match getin tomorrow very busy awareness a...   
1  some areas new england could see first flakes ...   
2  2nd worst qb definitely tony romo man likes sh...   
3  thailand washington us president barack obama ...   
4  did hear tony romo dressed up halloween giants...   

                                    stemmed_sentence  
0  won match getin tomorrow veri busi awar and de...  
1  some area new england could see first flake se...  
2  2nd worst qb definit toni romo man like share ...  
3  thailand washington us presid barack obama vow...  
4  did hear toni romo dress up halloween giant qu...

Let us check for the value distribution of the sentiment in our data

In [13]:
data_df["sentiment"].value_counts()

sentiment
neutral     6838
positive    5690
negative    2357
Name: count, dtype: int64

Let us convert the sentiment column to numerical (1 = Positive, 0 = Neutral, -1 = Negative)

In [14]:
data_df["sentiment"] = np.where((data_df["sentiment"] == "positive"), +1, data_df["sentiment"])
data_df["sentiment"] = np.where((data_df["sentiment"] == "negative"), -1, data_df["sentiment"])
data_df["sentiment"] = np.where((data_df["sentiment"] == "neutral"), 0, data_df["sentiment"])

In [15]:
data_df["sentiment"].value_counts()

sentiment
0     6838
1     5690
-1    2357
Name: count, dtype: int64

Final cleaned data

In [16]:
data_df.head()

sentiment                                           sentence  \
0         0  Won the match #getin . Plus\u002c tomorrow is ...   
1         0  Some areas of New England could see the first ...   
2        -1  @francesco_con40 2nd worst QB. DEFINITELY Tony...   
3         0  #Thailand Washington - US President Barack Oba...   
4         0  Did y\u2019all hear what Tony Romo dressed up ...   

                                    cleaned_sentence  \
0  won the match getin tomorrow is a very busy wi...   
1  some areas of new england could see the first ...   
2  2nd worst qb definitely tony romo the man who ...   
3  thailand washington us president barack obama ...   
4  did hear what tony romo dressed up as for hall...   

                           stopwords_remove_sentence  \
0  won match getin tomorrow very busy awareness a...   
1  some areas new england could see first flakes ...   
2  2nd worst qb definitely tony romo man likes sh...   
3  thailand washington us president barack obama ...   
4  did hear tony romo dressed up halloween giants...   

                                    stemmed_sentence  
0  won match getin tomorrow veri busi awar and de...  
1  some area new england could see first flake se...  
2  2nd worst qb definit toni romo man like share ...  
3  thailand washington us presid barack obama vow...  
4  did hear toni romo dress up halloween giant qu...

### Logistic Regression Model without feature engineering using  CountVectorizer

In [17]:
data_mod = data_df[["sentiment", "cleaned_sentence", "stopwords_remove_sentence", "stemmed_sentence"]]
data_mod.head()

sentiment                                   cleaned_sentence  \
0         0  won the match getin tomorrow is a very busy wi...   
1         0  some areas of new england could see the first ...   
2        -1  2nd worst qb definitely tony romo the man who ...   
3         0  thailand washington us president barack obama ...   
4         0  did hear what tony romo dressed up as for hall...   

                           stopwords_remove_sentence  \
0  won match getin tomorrow very busy awareness a...   
1  some areas new england could see first flakes ...   
2  2nd worst qb definitely tony romo man likes sh...   
3  thailand washington us president barack obama ...   
4  did hear tony romo dressed up halloween giants...   

                                    stemmed_sentence  
0  won match getin tomorrow veri busi awar and de...  
1  some area new england could see first flake se...  
2  2nd worst qb definit toni romo man like share ...  
3  thailand washington us presid barack obama vow...  
4  did hear toni romo dress up halloween giant qu...

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

Create a bag of words using count vectorizer

In [19]:
vectorizer_count = CountVectorizer(
    min_df = 3,
    ngram_range = (1, 1)
)

vectorizer_count2 = CountVectorizer(
    min_df = 3,
    ngram_range = (1, 1)
)

vectorizer_count3 = CountVectorizer(
    min_df = 3,
    ngram_range = (1, 1)
)

In [20]:
# Using stemmed sentence
features_stemmed = vectorizer_count.fit_transform(
    data_mod["stemmed_sentence"]
)

features_stemmed_nd = features_stemmed.toarray()

features_stopwords = vectorizer_count2.fit_transform(
    data_mod["stopwords_remove_sentence"]
)

features_stopwords_nd = features_stopwords.toarray()

features_cleaned = vectorizer_count3.fit_transform(
    data_mod["cleaned_sentence"]
)

features_cleaned_nd = features_cleaned.toarray()

In [21]:
data_labels = []
for i in data_mod["sentiment"]:
    data_labels.append(i)

In [22]:
from sklearn.model_selection import train_test_split

X_train_stemmed_cv, X_test_stemmed_cv, y_train_stemmed_cv, y_test_stemmed_cv  = train_test_split(
        features_stemmed_nd, 
        data_labels,
        train_size=0.80, 
        random_state=10)

X_train_stopwords_cv, X_test_stopwords_cv, y_train_stopwords_cv, y_test_stopwords_cv = train_test_split(
    features_stopwords_nd,
    data_labels,
    train_size=0.80,
    random_state=10
)

X_train_cleaned_cv, X_test_cleaned_cv, y_train_cleaned_cv, y_test_cleaned_cv = train_test_split(
    features_cleaned_nd,
    data_labels,
    train_size=0.80,
    random_state=10
)

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
log_model_1 = LogisticRegression(max_iter=1000)
log_model_2 = LogisticRegression(max_iter=1000)
log_model_3 = LogisticRegression(max_iter=1000)

In [25]:
log_model_1 = log_model_1.fit(X_train_stemmed_cv, y_train_stemmed_cv)
log_model_2 = log_model_2.fit(X_train_stopwords_cv, y_train_stopwords_cv)
log_model_3 = log_model_3.fit(X_train_cleaned_cv, y_train_cleaned_cv)

In [26]:
y_pred_cv1 = log_model_1.predict(X_test_stemmed_cv)
y_pred_cv2 = log_model_2.predict(X_test_stopwords_cv)
y_pred_cv3 = log_model_3.predict(X_test_cleaned_cv)

In [27]:
from sklearn.metrics import accuracy_score, classification_report

In [28]:
# Model 1
print(accuracy_score(y_test_stemmed_cv, y_pred_cv1))
print(classification_report(y_test_stemmed_cv, y_pred_cv1))

0.6741686261336917
              precision    recall  f1-score   support

          -1       0.61      0.45      0.51       483
           0       0.68      0.76      0.72      1361
           1       0.68      0.67      0.68      1133

    accuracy                           0.67      2977
   macro avg       0.66      0.62      0.64      2977
weighted avg       0.67      0.67      0.67      2977



In [29]:
# Model 2
print(accuracy_score(y_test_stopwords_cv, y_pred_cv2))
print(classification_report(y_test_stopwords_cv, y_pred_cv2))

0.6627477326167283
              precision    recall  f1-score   support

          -1       0.57      0.42      0.49       483
           0       0.67      0.75      0.71      1361
           1       0.69      0.66      0.67      1133

    accuracy                           0.66      2977
   macro avg       0.64      0.61      0.62      2977
weighted avg       0.66      0.66      0.66      2977



In [30]:
# Model 3
print(accuracy_score(y_test_cleaned_cv, y_pred_cv3))
print(classification_report(y_test_cleaned_cv, y_pred_cv3))

0.6677863621095063
              precision    recall  f1-score   support

          -1       0.60      0.43      0.50       483
           0       0.67      0.75      0.71      1361
           1       0.69      0.67      0.68      1133

    accuracy                           0.67      2977
   macro avg       0.65      0.62      0.63      2977
weighted avg       0.66      0.67      0.66      2977



### Logistic Regression Model without feature engineering using TFIDF Vectorizer

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
vectorizer_tfidf = TfidfVectorizer(min_df=5)

vectorizer_tfidf2 = TfidfVectorizer(min_df=5)

vectorizer_tfidf3 = TfidfVectorizer(min_df=5)

In [33]:
features_stemmed_tfidf = vectorizer_tfidf.fit_transform(data_mod["stemmed_sentence"])

features_stemmed_tfidf_nd = features_stemmed_tfidf.toarray()

features_stopwords_tfidf = vectorizer_tfidf2.fit_transform(data_mod["stopwords_remove_sentence"])

features_stopwords_tfidf_nd = features_stopwords_tfidf.toarray()

features_cleaned_tfidf = vectorizer_tfidf3.fit_transform(data_mod["cleaned_sentence"])

features_cleaned_tfidf_nd = features_cleaned_tfidf.toarray()


In [34]:
X_train_stemmed_tfidf, X_test_stemmed_tfidf, y_train_stemmed_tfidf, y_test_stemmed_tfidf  = train_test_split(
        features_stemmed_tfidf_nd, 
        data_labels,
        train_size=0.80, 
        random_state=10)

X_train_stopwords_tfidf, X_test_stopwords_tfidf, y_train_stopwords_tfidf, y_test_stopwords_tfidf  = train_test_split(
        features_stopwords_tfidf_nd, 
        data_labels,
        train_size=0.80, 
        random_state=10)

X_train_cleaned_tfidf, X_test_cleaned_tfidf, y_train_cleaned_tfidf, y_test_cleaned_tfidf  = train_test_split(
        features_cleaned_tfidf_nd, 
        data_labels,
        train_size=0.80, 
        random_state=10)

In [35]:
log_model_4 =  LogisticRegression()
log_model_5 =  LogisticRegression()
log_model_6 =  LogisticRegression()

In [36]:
log_model_4 = log_model_4.fit(X_train_stemmed_tfidf, y_train_stemmed_tfidf)
log_model_5 = log_model_5.fit(X_train_stopwords_tfidf, y_train_stopwords_tfidf)
log_model_6 = log_model_6.fit(X_train_cleaned_tfidf, y_train_cleaned_tfidf)

c:\Users\Adrian Jerez\Desktop\NLP_Project\NLP_Project\project_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Adrian Jerez\Desktop\NLP_Project\NLP_Project\project_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https

In [37]:
y_pred_stemmed_tfidf = log_model_4.predict(X_test_stemmed_tfidf)
print(accuracy_score(y_test_stemmed_tfidf, y_pred_stemmed_tfidf))
print(classification_report(y_test_stemmed_tfidf, y_pred_stemmed_tfidf))

0.6751763520322472
              precision    recall  f1-score   support

          -1       0.70      0.34      0.45       483
           0       0.66      0.81      0.73      1361
           1       0.70      0.65      0.68      1133

    accuracy                           0.68      2977
   macro avg       0.68      0.60      0.62      2977
weighted avg       0.68      0.68      0.66      2977



In [38]:
y_pred_stopwords_tfidf = log_model_5.predict(X_test_stopwords_tfidf)
print(accuracy_score(y_test_stopwords_tfidf, y_pred_stopwords_tfidf))
print(classification_report(y_test_stopwords_tfidf, y_pred_stopwords_tfidf))

0.6738327175008397
              precision    recall  f1-score   support

          -1       0.68      0.31      0.43       483
           0       0.65      0.82      0.72      1361
           1       0.71      0.65      0.68      1133

    accuracy                           0.67      2977
   macro avg       0.68      0.59      0.61      2977
weighted avg       0.68      0.67      0.66      2977



In [39]:
y_pred_cleaned_tfidf = log_model_6.predict(X_test_cleaned_tfidf)
print(accuracy_score(y_test_cleaned_tfidf, y_pred_cleaned_tfidf))
print(classification_report(y_test_cleaned_tfidf, y_pred_cleaned_tfidf))

0.6731609002351361
              precision    recall  f1-score   support

          -1       0.69      0.33      0.44       483
           0       0.65      0.82      0.72      1361
           1       0.71      0.65      0.68      1133

    accuracy                           0.67      2977
   macro avg       0.68      0.60      0.61      2977
weighted avg       0.68      0.67      0.66      2977



### Naive Bayes Model without Feature Engineering

In [40]:
from sklearn.naive_bayes import MultinomialNB

#### Using CountVectorizer

In [41]:
multinomialNB = MultinomialNB(force_alpha=True)

multinomialNB.fit(X_train_stopwords_cv, y_train_stopwords_cv)

MultinomialNB(force_alpha=True)

In [42]:
y_pred_nb_cv = multinomialNB.predict(X_test_stopwords_cv)
print(accuracy_score(y_test_stopwords_cv, y_pred_nb_cv))

0.6341954988243198


Using TF-IDF Vectorizer

In [43]:
multinomialNB2 = MultinomialNB(force_alpha=True)

multinomialNB2.fit(X_train_stopwords_tfidf, y_train_stopwords_tfidf)

y_pred_nb_tfidf = multinomialNB2.predict(X_test_stopwords_tfidf)
print(accuracy_score(y_test_stopwords_tfidf, y_pred_nb_tfidf))

0.6234464225730602


### Logistic Regression Models with Oversampling

In [44]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

In [45]:
oversample = RandomOverSampler(sampling_strategy='minority')

X_over, y_over = oversample.fit_resample(features_stopwords_nd, data_labels)
X_over, y_over = oversample.fit_resample(X_over, y_over)
print(Counter(y_over))

Counter({0: 6838, -1: 6838, 1: 6838})


In [46]:
X_train_os, X_test_os, y_train_os, y_test_os  = train_test_split(
        X_over, 
        y_over,
        train_size=0.80, 
        random_state=10)

In [47]:
log_model_7 = LogisticRegression(max_iter=1000)
log_model_7 = log_model_7.fit(X_train_os, y_train_os)

In [48]:
y_pred = log_model_7.predict(X_test_os)
print(accuracy_score(y_test_os, y_pred))
print(classification_report(y_test_os, y_pred))

0.7652936875456983
              precision    recall  f1-score   support

          -1       0.81      0.89      0.85      1411
           0       0.70      0.68      0.69      1318
           1       0.78      0.71      0.74      1374

    accuracy                           0.77      4103
   macro avg       0.76      0.76      0.76      4103
weighted avg       0.76      0.77      0.76      4103



Lets apply GridSearch to get the best hyperparameter

In [ ]:
# from sklearn.model_selection import GridSearchCV

# parameters = {
#     'C': np.logspace(-5, 8, 15),
#     'max_iter': [1,2,3,4,5,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150]
# }

# grid_search = LogisticRegression()

# grid_search = GridSearchCV(
#     grid_search,
#     parameters,
#     cv = 5,
#     scoring = 'accuracy',
#     n_jobs=-1
# )

# grid_result = grid_search.fit(X_train, y_train)

# print("Best parameters: ", grid_result.best_params_)

In [49]:
log_model_8 = LogisticRegression(C = 3.727593720314938, max_iter=40)
log_model_8.fit(X_train_os, y_train_os)

c:\Users\Adrian Jerez\Desktop\NLP_Project\NLP_Project\project_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=3.727593720314938, max_iter=40)

In [50]:
y_pred = log_model_8.predict(X_test_os)
print(accuracy_score(y_test_os, y_pred))
print(classification_report(y_test_os, y_pred))

0.7567633438947112
              precision    recall  f1-score   support

          -1       0.80      0.87      0.84      1411
           0       0.68      0.69      0.69      1318
           1       0.78      0.70      0.74      1374

    accuracy                           0.76      4103
   macro avg       0.75      0.75      0.75      4103
weighted avg       0.76      0.76      0.76      4103



### Naive Bayes with Oversampling

In [51]:
multinomialNB_os = MultinomialNB(force_alpha=True)

multinomialNB_os.fit(X_train_os, y_train_os)

MultinomialNB(force_alpha=True)

In [52]:
y_pred_nb_os = multinomialNB_os.predict(X_test_os)
print(accuracy_score(y_test_os, y_pred_nb_os))
print(classification_report(y_test_os, y_pred_nb_os))

0.7033877650499635
              precision    recall  f1-score   support

          -1       0.75      0.82      0.78      1411
           0       0.66      0.58      0.62      1318
           1       0.69      0.70      0.70      1374

    accuracy                           0.70      4103
   macro avg       0.70      0.70      0.70      4103
weighted avg       0.70      0.70      0.70      4103



### Models using cleaned sentence and nltk.stopwords

In [53]:
from nltk.corpus import stopwords

vectorizer_tfidf = TfidfVectorizer(min_df=5, max_df=0.8, max_features=2500, stop_words = stopwords.words('english'))

features_cleaned_tfidf_2 = vectorizer_tfidf.fit_transform(data_mod["cleaned_sentence"])

features_cleaned_tfidf_2_nd = features_cleaned_tfidf_2.toarray()


In [54]:
X_over, y_over = oversample.fit_resample(features_cleaned_tfidf_2_nd, data_labels)
X_over, y_over = oversample.fit_resample(X_over, y_over)
print(Counter(y_over))

Counter({0: 6838, -1: 6838, 1: 6838})


In [55]:
X_train_cleaned_tfidf_2, X_test_cleaned_tfidf_2, y_train_cleaned_tfidf_2, y_test_cleaned_tfidf_2  = train_test_split(
        X_over, 
        y_over,
        train_size=0.80, 
        random_state=10)

In [56]:
log_model_9 = LogisticRegression(max_iter=1000)
log_model_9 = log_model_9.fit(X_train_cleaned_tfidf_2, y_train_cleaned_tfidf_2)

In [57]:
y_pred = log_model_9.predict(X_test_cleaned_tfidf_2)
print(accuracy_score(y_test_cleaned_tfidf_2, y_pred))
print(classification_report(y_test_cleaned_tfidf_2, y_pred))

0.7007067999025104
              precision    recall  f1-score   support

          -1       0.73      0.76      0.75      1411
           0       0.63      0.68      0.66      1318
           1       0.75      0.66      0.70      1374

    accuracy                           0.70      4103
   macro avg       0.70      0.70      0.70      4103
weighted avg       0.70      0.70      0.70      4103



### Saving models to a File Format using JobLib

In [58]:
import joblib

In [59]:
model_filename = "log_reg_oversampling_stopwords_78acc.joblib"
vectorizer_filename = "count_vectorizer.joblib"
joblib.dump(log_model_7, "nlp_project/models/" + model_filename)
joblib.dump(vectorizer_count2, "nlp_project/models/" + vectorizer_filename)

['nlp_project/models/count_vectorizer.joblib']